In [1]:
import torch.nn   as nn
import GM_library as GML
import numpy      as np
import torch
import json

from os                   import path, listdir
from torch_geometric.data import Data

import sys
sys.path.append('../')
import MP.MP_library as MPL

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Target to generate new crystals
target = 'GM_BiSI'

input_folder    = 'models'
target_folder   = f'{input_folder}/{target}'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

In [7]:
# Read the file in JSON format to a dictionary
with open(f'{target_folder}/model_parameters.json', 'r') as json_file:
    model_parameters = json.load(json_file)

# Number of graphs to predict
N_predictions = 10

# Number of diffusing and denoising steps
n_t_steps = model_parameters['n_t_steps']

# Decay of parameter alpha
noise_contribution = model_parameters['noise_contribution']
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = model_parameters['dropout_node']
dropout_edge = model_parameters['dropout_edge']

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [9]:
labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.pt'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
dataset    = torch.load(dataset_name_std)
parameters = torch.load(parameters_name_std)

# Assigning parameters accordingly
target_mean, feat_mean, edge_mean, target_std, edge_std, feat_std, scale = parameters

# Defining target factor
target_factor = target_std / scale

In [10]:
# Calculate the mean and standard deviation of the number of nodes
total_nodes = torch.tensor([data.num_nodes for data in dataset])
mean_nodes  = torch.mean(total_nodes.float()).item()
std_nodes   = torch.std(total_nodes.float()).item()

mean_nodes, std_nodes

(71.86479949951172, 0.34193673729896545)

# Loading the model

In [11]:
# Determine number of features in dataset, considering the t_step information
n_features = dataset[0].num_node_features + 1

# Instantiate the models for nodes and edges
node_model = GML.nGCNN(n_features, dropout_node).to(device)
node_model.load_state_dict(torch.load(node_model_name))
node_model.eval()

edge_model = GML.eGCNN(n_features, dropout_edge).to(device)
edge_model.load_state_dict(torch.load(edge_model_name))
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)


Node GCNN:
nGCNN(
  (conv1): GraphConv(5, 256)
  (conv2): GraphConv(256, 5)
)

Edge GCNN:
eGCNN(
  (linear1): Linear(in_features=6, out_features=32, bias=True)
  (linear2): Linear(in_features=32, out_features=1, bias=True)
)


# Generating new cystals

In [12]:
# Predicting loop
predicted_dataset = []
with torch.no_grad():
    for i in range(N_predictions):
        # Get random number of nodes
        n_nodes = int(np.random.normal(mean_nodes, std_nodes))
        
        # Get random graph, acting as diffused
        diffused_graph = GML.get_random_graph(n_nodes, n_features-1)
        
        # Denoise the diffused graph
        #print(f'Denoising...')
        denoised_graph, _ = GML.denoise(diffused_graph, n_t_steps, node_model, edge_model,
                                        s=alpha_decay)
        
        # Append generated graph
        predicted_dataset.append(denoised_graph)

# Revert stardadization
denoised_graphs = GML.revert_standardize_dataset(predicted_dataset, parameters)
denoised_graphs

[Data(x=[71, 4], edge_index=[2, 2485], edge_attr=[2485]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[71, 4], edge_index=[2, 2485], edge_attr=[2485]),
 Data(x=[71, 4], edge_index=[2, 2485], edge_attr=[2485]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[71, 4], edge_index=[2, 2485], edge_attr=[2485]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556]),
 Data(x=[71, 4], edge_index=[2, 2485], edge_attr=[2485]),
 Data(x=[72, 4], edge_index=[2, 2556], edge_attr=[2556])]

In [13]:
for i in range(N_predictions):
    print()
    print(i+1)
    graph = denoised_graphs[i].clone()
    try:
        GML.POSCAR_graph_encoding(graph, L, file_name=f'POSCAR-{i}', POSCAR_directory='./')
    except SystemExit:
        continue


1
25 30 68 11.714252 10.226208 10.617725 5.508799806263361 8.615593380385214

2
31 37 66 10.806225 10.929033 11.29813 5.023517594227143 9.7060825982029

3
49 59 61 10.358076 11.801014 10.567783 6.510650446581332 9.842528177694057

4
16 35 68 10.447621 10.900649 10.503468 5.630664974911289 9.333796767860683

5
3 50 61 11.09833 10.609603 10.661455 5.499474784362518 9.073006744702468

6
29 43 66 10.282546 10.72827 11.4563265 4.35588152731182 9.804186056852101

7
0 32 39 10.590068 10.8727045 10.337479 5.831019103622349 9.176868719026833

8
1 28 32 10.434946 10.515156 11.060737 4.653435900685312 9.429423930082594

9
38 39 49 10.692003 9.641688 11.845575 3.1314727067913575 9.118992974262596

10
5 25 51 11.181639 10.61794 10.982516 5.238678877884309 9.235631671723157
